In [1]:
import csv
import datetime
import matplotlib.pyplot as plt
import nltk
from time import time
import joblib
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import plot_confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn import tree
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, precision_recall_fscore_support
from sklearn.model_selection import cross_validate
pd.set_option('display.max_rows', 10000)

In [2]:
#Stemmer
italian_stemmer = SnowballStemmer('italian')
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([italian_stemmer.stem(w) for w in analyzer(doc)])

In [3]:
#Data validation
def validate(date_text):
    try:
        datetime.datetime.strptime(date_text, '%Y-%m-%d')
    except ValueError:
        raise ValueError("Incorrect data format, should be YYYY-MM-DD")

In [4]:
#Benchmark
def benchmark(data_label, data_predicted):
    accuracy=np.mean(data_predicted == data_label)#accuracy extreaction
    print('accuracy : ' + str(accuracy))

    f_score = f1_score(data_label, data_predicted, average='macro')
    print('f_score : ' + str(f_score) + '\n')

    conf_matrix = confusion_matrix(data_label, data_predicted, labels = [0,1,2], normalize = 'true')
    print(conf_matrix)
    result = precision_recall_fscore_support(data_label,data_predicted)
    print ("Precision,Recall,f_score\n", 
       "Classe 0: ", result[0], "\n",
       "Classe 1: ", result[1], "\n",
       "Classe 2: ", result[2], "\n",
       "Support:  ", result[3])

In [5]:
def search_tweet_Period(df:pd.DataFrame, start_date = "2021-02-01", end_date = "2021-05-31"):
    
    validate(start_date)
    validate(end_date)
    
    return df[(df['timestamp'] > start_date) & (df['timestamp'] < end_date)]

In [6]:
def OpenDataset(filename):
    #Load data and a short phase of preprocess
    test_set = pd.read_csv(filename,sep=',',usecols=['timestamp','username','likes_count','tweet'])
    test_set.drop_duplicates(subset=['tweet'])
    test_set = test_set.dropna()
    
    news = []
    with open('AccountToDelete.txt') as f:
        for line in f:
            news.append(line.strip())
    
    indexNames = test_set[ test_set['username'].isin(news)].index
    test_set.drop(indexNames, inplace=True)
    
    return test_set